<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите тетрадку в соответствующем Peer Review. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1pLsegkAICL9PzOLyAeH9DmDOBfktte0l8JW75uWcTng). 

# Решение

Дальше будет много попыток получить лучшую метрику, но чтобы проверяющему не читать это всё сделаю краткую навигацию по основным пунктам программы.

бенчмарки:
* **A3 strong baseline (20 credits)** -- 0.95965
* **A3 strong baseline (10 credits)** -- 0.95343
* **Logit Tf-Idf 6 features** -- 0.95216
* **<font color='red'>Итоговый результат (n-surkov)</font>** -- 0.94171
* **Logit +3 features** и **A3 baseline 2** -- 0.92784
* **CountVectorizer-logit-3feat** -- 0.92692
* **CountVectorizer-logit-3feat** -- 0.92692
* **SGDClassifier** -- 0.91273

**Критерии оценки работы:**
* Правильные ли получились размерности матриц в п. 1? ([ответ](#answer1))
* Правильным ли получилось значения ROC AUC в п. 2? ([ответ](#answer2))
* Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? ([лучший результат](#cv+2fext_result) **0.94171**)
* Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? ([лучший результат](#cv+2fext_result) **0.94171**)

**ник на kaggle n-surkov**

**Навигация по решению**
* [Подготовка данных](#data_loading)
* [Применение CountVectorizer](#count_vectorizer) ([полученный результат](#count_vectirizer_result))
* [Добавление первой фичи](#cv+1fext) ([полученный результат](#cv+1fext_result))
* [Добавление второй фичи](#cv+2fext) ([полученный результат](#cv+2fext_result))
* [Дальнейшее неудачное исследование](#other) в которое вошли попытки добавить посещаемость некоторых сайтов, продолжительность сессии и настроить параметры слассификаторов

In [1]:
%load_ext autoreload
%autoreload 2

In [332]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV, LogisticRegression
from sklearn.metrics import roc_auc_score
from collections import Counter
import scipy.sparse
from sklearn.svm import LinearSVC
# полдключаем библиотеку проекта
import sys
sys.path.insert(1, './../src/')
from preprocessing import *

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [3]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = os.path.join('.', '..', 'data', 'kaggle')

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [4]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [7]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [9]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [5]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [11]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [6]:
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as fo:
    site_dic = pickle.load(fo)
site_freq = dict([(key, (val, )) for key, val in site_dic.items()])
with open(os.path.join(PATH_TO_DATA, 'site_freq.pkl'), 'wb') as fo:
    pickle.dump(site_freq, fo)

In [7]:
train_test_sparse = make_csr_matrix(train_test_df_sites.to_numpy(), site_freq)

In [8]:
X_train_sparse = train_test_sparse[:253561, :]
X_test_sparse = train_test_sparse[253561:, :]
y = train_df.target.to_numpy()

**<font color='red'>Вопрос 1. </font> Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [15]:
print('X_train_sparce shape is {} according to train_df shape = {}'.format(X_train_sparse.shape, train_df.shape))
print('X_test_sparce shape is {} according to test_df shape = {}'.format(X_test_sparse.shape, test_df.shape))
print(X_train_sparse.shape[0], X_train_sparse.shape[1], X_test_sparse.shape[0], X_test_sparse.shape[1])

X_train_sparce shape is (253561, 48371) according to train_df shape = (253561, 21)
X_test_sparce shape is (82797, 48371) according to test_df shape = (82797, 20)
253561 48371 82797 48371


<a id='answer1'></a>
**Итого получаем число строк и столбцов матрицы X_train_sparse, затем число строк и столбцов матрицы X_test_sparse:**

<font color='red'>**253561 48371 82797 48371**</font>

[К решению](#Решение)

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [16]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [9]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [17]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)

CPU times: user 410 ms, sys: 156 ms, total: 566 ms
Wall time: 321 ms


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=17, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [18]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

**<font color='red'>Вопрос 2. </font> Посчитайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [19]:
roc_auc = roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])

In [20]:
print(roc_auc)

0.9336212361738347


<a id='answer2'></a>
**ROC AUC логистической регрессии**

<font color='red'>**0.934**</font>

[К решению](#Решение)

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [21]:
%%time
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

CPU times: user 499 ms, sys: 179 ms, total: 678 ms
Wall time: 430 ms


**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [4]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [23]:
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join('answers', 'SGD_first_submission.csv'))

Получил Score = 0.91646

<a id='data_loading'></a>
## Подготовка данных

[К решению](#Решение)

Далее была собрана обучающая выборка из 150 случайных пользователей по данным соревнования. Загрузим её.

In [5]:
time_cols = ['time%d' % i for i in range(1, 11)]
train_subdf = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions_u150.csv'),
                          index_col='session_id',
                          parse_dates=time_cols)

X_train_df, X_valid_df, y_train_ss, y_valid_ss = train_test_split(train_subdf, 
                                                                  train_subdf.target, 
                                                                  test_size=0.33, shuffle=True, 
                                                                  stratify=train_subdf.target,
                                                                  random_state=17)
sites_cols = ['site%d' % i for i in range(1, 11)]
X_train_ss = X_train_df[sites_cols].fillna(0).astype('int')
X_valid_ss = X_valid_df[sites_cols].fillna(0).astype('int')

In [6]:
with open(os.path.join(PATH_TO_DATA, 'site_freq.pkl'), 'rb') as fo:
    site_freq = pickle.load(fo)
X_train_sparse_ss = make_csr_matrix(X_train_ss.to_numpy(), site_freq)
X_valid_sparse_ss = make_csr_matrix(X_valid_ss.to_numpy(), site_freq)

И подгрузим нормально исходные данные.

In [7]:
time_cols = ['time%d' % i for i in range(1, 11)]
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                          index_col='session_id',
                          parse_dates=time_cols)
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                          index_col='session_id',
                          parse_dates=time_cols)

Получим на данной тестовой выборке метрику логистической регрессии.

In [8]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_sparse_ss, y_train_ss)
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid_sparse_ss)
roc_auc = roc_auc_score(y_valid_ss, logit_valid_pred_proba[:, 1])
print('ROC AUC for X_sparse with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for X_sparse with SGDClassifier is 0.96812


Метрика получилась выше, но это и не важно, нам она нужна для сравнения алгоритмов между собой.

<a id='count_vectorizer'></a>
## Попробуем логистическую регрессию с CountVectorizer

[К решению](#Решение)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [59]:
print('sessions have {} unique sites'.format(len(site_freq)))

sessions have 48371 unique sites


Примем 50к фичей и включим 1,2,3-граммы

In [14]:
def prepare_data_countvec(X_train, X_test):
    vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=50000)
    X_train = X_train[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
    X_test = X_test[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
    
    X_train.to_csv('X_train_text.txt', sep=' ', index=None, header=None)
    X_test.to_csv('X_test_text.txt', sep=' ', index=None, header=None)
    
    with open('X_train_text.txt', 'r') as fo:
        X_train = vectorizer.fit_transform(fo)
    with open('X_test_text.txt', 'r') as fo:
        X_test = vectorizer.transform(fo)
    
    return X_train, X_test

In [15]:
X_train_cv, X_test_cv = prepare_data_countvec(X_train_ss, X_valid_ss)

In [16]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, y_train_ss)
logit_valid_pred_proba = sgd_logit.predict_proba(X_test_cv)
roc_auc = roc_auc_score(y_valid_ss, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer with SGDClassifier is 0.96456


<a id='count_vectorizer_result'></a>
**Получилось практически то же самое. Правда на реальных данных метрика получилась чуть выше (0.91789), но тоже несущественно.**

## Попробуем логистическую регрессию с TfIdfVEctorizer

In [17]:
def prepare_data_tfidfvec(X_train, X_test):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=50000)
    X_train = X_train[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
    X_test = X_test[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
    
    X_train.to_csv('X_train_text.txt', sep=' ', index=None, header=None)
    X_test.to_csv('X_test_text.txt', sep=' ', index=None, header=None)
    
    with open('X_train_text.txt', 'r') as fo:
        X_train = vectorizer.fit_transform(fo)
    with open('X_test_text.txt', 'r') as fo:
        X_test = vectorizer.transform(fo)
    
    return X_train, X_test

In [102]:
X_train_cv, X_test_cv = prepare_data_tfidfvec(X_train_ss, X_valid_ss)

In [103]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, y_train_ss)
logit_valid_pred_proba = sgd_logit.predict_proba(X_test_cv)
roc_auc = roc_auc_score(y_valid_ss, logit_valid_pred_proba[:, 1])
print('ROC AUC for TfidfVectorizer with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for TfidfVectorizer with SGDClassifier is 0.95506


**Получилось меньше, чем у CounterVectorizer, отправлять нет смысла**

**Попробуем LinearCSV, с параметрами, настроенными на 4 неделе.**

In [104]:
from sklearn.linear_model import LogisticRegression

In [105]:
X_train_cv, X_test_cv = prepare_data_countvec(X_train_ss, X_valid_ss)

In [106]:
estimator = LogisticRegression(C=2.9050786505108603, n_jobs=-1, random_state=17)
estimator.fit(X_train_cv, y_train_ss)
logit_valid_pred_proba = estimator.predict_proba(X_test_cv)
roc_auc = roc_auc_score(y_valid_ss, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer with LogistticRegression is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer with LogistticRegression is 0.96945


**Значительных улучшений не дало, пробуем искать дальше**

<a id='cv+1fext'></a>
## Дополним данные временем суток начала сессии

[К решению](#Решение)

In [9]:
def extend_features(csr, X):
    features = [
                'daily_aсtivity', 
               ]
    sessions = X
    sessions['user_id'] = np.zeros((len(X), ), dtype=int)
    X_ext = prepare_train_set_fe(sessions,
                                 site_freq_path=os.path.join(PATH_TO_DATA, 'site_freq.pkl'),
                                 feature_names=features)
    
    csr_ext = csr.copy()
    for t in np.unique(X_ext['daily_aсtivity']):
        new_data = (X_ext['daily_aсtivity'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
    
    return csr_ext
    
def prepare_data_ext(X_train, X_test):
    vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=50000)
    
    X_train_ss = X_train[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
    X_test_ss = X_test[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
    
    X_train_ss.to_csv('X_train_text.txt', sep=' ', index=None, header=None)
    X_test_ss.to_csv('X_test_text.txt', sep=' ', index=None, header=None)
    
    with open('X_train_text.txt', 'r') as fo:
        X_train_csr = vectorizer.fit_transform(fo)
    with open('X_test_text.txt', 'r') as fo:
        X_test_csr = vectorizer.transform(fo)
        
    X_train = extend_features(X_train_csr, X_train)
    X_test = extend_features(X_test_csr, X_test)
    
    return X_train, X_test

In [124]:
X_train_cv, X_test_cv = prepare_data_ext(X_train_df, X_valid_df)

In [125]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, y_train_ss)
logit_valid_pred_proba = sgd_logit.predict_proba(X_test_cv)
roc_auc = roc_auc_score(y_valid_ss, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 1 feature with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 3 features with SGDClassifier is 0.98094


**Значительно лучше, попробуем на полной тестовой выборке.**

In [136]:
X_train_cv, X_test_cv = prepare_data_ext(train_df.iloc[:, :-1], test_df.iloc[:, :-1])
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_cv.tocsr()[:train_share, :], train_df.target[:train_share]
X_valid, y_valid  = X_train_cv.tocsr()[train_share:, :], train_df.target[train_share:]

In [139]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
roc_auc = roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 1 feature with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 1 feature with SGDClassifier is 0.96457


Действительно лучше, можно попробовать на реальных данных.

In [141]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, train_df.target)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_cv)
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join('answers', 'SGD_countVect_1fext.csv'))

<a id='cv+1fext_result'></a>
**Получил Score = 0.93744**

<a id='cv+2fext'></a>
## Добавление дня недели начала сессии

[К решению](#Решение)

In [20]:
def extend_features(csr, X):
    features = [
                'day_of_week', 
                'daily_aсtivity',
               ]
    sessions = X
    sessions['user_id'] = np.zeros((len(X), ), dtype=int)
    X_ext = prepare_train_set_fe(sessions,
                                 site_freq_path=os.path.join(PATH_TO_DATA, 'site_freq.pkl'),
                                 feature_names=features)
    
    csr_ext = csr.copy()
    for t in np.unique(X_ext['daily_aсtivity']):
        new_data = (X_ext['daily_aсtivity'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    for t in np.unique(X_ext['day_of_week']):
        new_data = (X_ext['day_of_week'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    return csr_ext

In [21]:
X_train_cv, X_test_cv = prepare_data_ext(X_train_df, X_valid_df)

In [22]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, y_train_ss)
logit_valid_pred_proba = sgd_logit.predict_proba(X_test_cv)
roc_auc = roc_auc_score(y_valid_ss, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 2 features with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 4 features with SGDClassifier is 0.98489


Ещё лучше, пробуем на полной.

In [153]:
X_train_cv, X_test_cv = prepare_data_ext(train_df.iloc[:, :-1], test_df.iloc[:, :-1])
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_cv.tocsr()[:train_share, :], train_df.target[:train_share]
X_valid, y_valid  = X_train_cv.tocsr()[train_share:, :], train_df.target[train_share:]

In [154]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
roc_auc = roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 4 feature with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 4 feature with SGDClassifier is 0.96964


Можно пробовать отправить

In [155]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, train_df.target)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_cv)
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join('answers', 'SGD_countVect_4fext.csv'))

<a id='cv+2fext_result'></a>
**Получил Score = 0.94171**

<a id='other'></a>
## Остальные неудачные попытки

Добавим продолжительность сессии в признаки.

In [286]:
def extend_features(csr, X):
    features = [
#                 '#unique_sites',
                'session_timespan', 
#                 'start_hour',
                'day_of_week', 
#                 'timespan_median', 
#                 'timespan_mean',
                'daily_aсtivity', 
#                 'freq_facebook', 
#                 'timespan_youtube',
#                 'timespan_mail', 
#                 'freq_googlevideo', 
#                 'freq_google',
# #                 'live',
# #                 'trouter',
#                 'skype',
#                 'twitter',
               ]
    sessions = X
    sessions['user_id'] = np.zeros((len(X), ), dtype=int)
    X_ext = prepare_train_set_fe(sessions,
                                 site_freq_path=os.path.join(PATH_TO_DATA, 'site_freq.pkl'),
                                 feature_names=features)
    
    csr_ext = csr.copy()
    for t in np.unique(X_ext['daily_aсtivity']):
        new_data = (X_ext['daily_aсtivity'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    for t in np.unique(X_ext['day_of_week']):
        new_data = (X_ext['day_of_week'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
    
    bm = [-1, 60, 5*60, 10*60, 30*60]
    for i in range(len(bm) - 1):
        new_data = ((X_ext['session_timespan'] > bm[i]) & \
                    (X_ext['session_timespan'] <= bm[i+1])).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    return csr_ext

In [292]:
X_train_cv, X_test_cv = prepare_data_ext(X_train_df, X_valid_df)

In [293]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, y_train_ss)
logit_valid_pred_proba = sgd_logit.predict_proba(X_test_cv)
roc_auc = roc_auc_score(y_valid_ss, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 3 features with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 3 features with SGDClassifier is 0.98492


Метрика получше, посмотрим, как на полной тестовой выборке.

In [294]:
X_train_cv, X_test_cv = prepare_data_ext(train_df.iloc[:, :-1], test_df.iloc[:, :-1])
train_share = int(.7 * len(train_df))
X_train, y_train = X_train_cv.tocsr()[:train_share, :], train_df.target[:train_share]
X_valid, y_valid  = X_train_cv.tocsr()[train_share:, :], train_df.target[train_share:]

In [295]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
roc_auc = roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 3 feature with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 3 feature with SGDClassifier is 0.96997


Действительно получшеб попробуем отправить.

In [296]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, train_df.target)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_cv)
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join('answers', 'SGD_countVect_3fext.csv'))

Получил Score = 0.94169 -- копание на месте.

Попробуем внедрить индикацию посещений некоторых сайтов.

In [16]:
sorted_site_freq = sorted(site_freq.items(), key=lambda x: x[1])
sorted_site_freq

[('fpdownload2.macromedia.com', (1,)),
 ('hotmail.fr', (2,)),
 ('login.live.com', (3,)),
 ('mail.live.com', (4,)),
 ('dub122.mail.live.com', (5,)),
 ('people.directory.live.com', (6,)),
 ('secure.shared.live.com', (7,)),
 ('windowslive.tt.omtrdc.net', (8,)),
 ('cid-1bed360223325286.users.storage.live.com', (9,)),
 ('js.live.net', (10,)),
 ('go.trouter.io', (11,)),
 ('storage.live.com', (12,)),
 ('blufiles.storage.msn.com', (13,)),
 ('h.live.com', (14,)),
 ('prod.registrar.skype.com', (15,)),
 ('api.skype.com', (16,)),
 ('proxy-bay-people.directory.live.com', (17,)),
 ('p.sfx.ms', (18,)),
 ('dub122.afx.ms', (19,)),
 ('google.fr', (20,)),
 ('www.google.fr', (21,)),
 ('apis.google.com', (22,)),
 ('www.google.com', (23,)),
 ('lipsakiss.free.fr', (24,)),
 ('www.shotgun2013.fr', (25,)),
 ('ba.commentcamarche.net', (26,)),
 ('www.commentcamarche.net', (27,)),
 ('static.commentcamarche.net', (28,)),
 ('www.facebook.com', (29,)),
 ('platform.twitter.com', (30,)),
 ('static.ccm2.net', (31,)),
 (

Попробуем разделить по пользованию skype, facebook и twitter

In [297]:
def extend_features(csr, X):
    features = [
                'session_timespan', 
                'day_of_week', 
                'daily_aсtivity', 
                'freq_facebook', 
                'skype',
                'twitter',
               ]
    sessions = X
    sessions['user_id'] = np.zeros((len(X), ), dtype=int)
    X_ext = prepare_train_set_fe(sessions,
                                 site_freq_path=os.path.join(PATH_TO_DATA, 'site_freq.pkl'),
                                 feature_names=features)
    
    csr_ext = csr.copy()
    for t in np.unique(X_ext['daily_aсtivity']):
        new_data = (X_ext['daily_aсtivity'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    for t in np.unique(X_ext['day_of_week']):
        new_data = (X_ext['day_of_week'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
    
    bm = [-1, 60, 5*60, 10*60, 30*60]
    for i in range(len(bm) - 1):
        new_data = ((X_ext['session_timespan'] > bm[i]) & \
                    (X_ext['session_timespan'] <= bm[i+1])).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    csr_ext = hstack([csr_ext, X_ext['freq_facebook'].to_numpy().reshape(-1, 1)])
    csr_ext = hstack([csr_ext, X_ext['skype'].to_numpy().reshape(-1, 1)])
    csr_ext = hstack([csr_ext, X_ext['twitter'].to_numpy().reshape(-1, 1)])
        
    return csr_ext

In [300]:
X_train_cv, X_test_cv = prepare_data_ext(train_df.iloc[:, :-1], test_df.iloc[:, :-1])
train_share = int(.7 * len(train_df))
X_train, y_train = X_train_cv.tocsr()[:train_share, :], train_df.target[:train_share]
X_valid, y_valid  = X_train_cv.tocsr()[train_share:, :], train_df.target[train_share:]

In [302]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
roc_auc = roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 6 feature with SGDClassifier is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 6 feature with SGDClassifier is 0.96999


In [303]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_cv, train_df.target)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_cv)
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join('answers', 'SGD_countVect_6fext.csv'))

Получил Score = 0.94045. Ещё меньше. В общем пока оставляем вариант с 2 признаками и покрутим параметры самой модели.

In [304]:
def extend_features(csr, X):
    features = [
                'day_of_week', 
                'daily_aсtivity',
               ]
    sessions = X
    sessions['user_id'] = np.zeros((len(X), ), dtype=int)
    X_ext = prepare_train_set_fe(sessions,
                                 site_freq_path=os.path.join(PATH_TO_DATA, 'site_freq.pkl'),
                                 feature_names=features)
    
    csr_ext = csr.copy()
    for t in np.unique(X_ext['daily_aсtivity']):
        new_data = (X_ext['daily_aсtivity'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    for t in np.unique(X_ext['day_of_week']):
        new_data = (X_ext['day_of_week'] == t).astype('int')
        csr_ext = hstack([csr_ext, new_data.to_numpy().reshape(-1, 1)])
        
    return csr_ext

In [305]:
X_train_cv, X_test_cv = prepare_data_ext(X_train_df, X_valid_df)

Попробуем логистическую регрессию.

In [313]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

In [330]:
%%time
logit_c_values = np.linspace(5, 10, 20)
logit_grid_searcher = LogisticRegressionCV(logit_c_values, random_state=17, cv=skf, n_jobs=-1)
logit_grid_searcher.fit(X_train_cv, y_train_ss)

CPU times: user 1.17 s, sys: 1.13 s, total: 2.3 s
Wall time: 14.5 s


LogisticRegressionCV(Cs=array([ 5.        ,  5.26315789,  5.52631579,  5.78947368,  6.05263158,
        6.31578947,  6.57894737,  6.84210526,  7.10526316,  7.36842105,
        7.63157895,  7.89473684,  8.15789474,  8.42105263,  8.68421053,
        8.94736842,  9.21052632,  9.47368421,  9.73684211, 10.        ]),
                     class_weight=None,
                     cv=StratifiedKFold(n_splits=3, random_state=17, shuffle=True),
                     dual=False, fit_intercept=True, intercept_scaling=1.0,
                     l1_ratios=None, max_iter=100, multi_class='warn',
                     n_jobs=-1, penalty='l2', random_state=17, refit=True,
                     scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [331]:
logit_cv_scores = list(logit_grid_searcher.scores_.values())[0]
logit_mean_cv_scores = logit_cv_scores.mean(axis=0)
assert (logit_grid_searcher.Cs_ == logit_c_values).all()
i = np.argmax(logit_mean_cv_scores)
accu_train = logit_mean_cv_scores[i]
print('best mean score is {:.5f} correspond to C={} with index {}'.format(logit_mean_cv_scores[i], logit_c_values[i], i))

best mean score is 0.96453 correspond to C=7.631578947368421 with index 10


Не похоже на то, чтобы что-то улучшилось. Попробуем ещё опорные вектора

In [328]:
%%time
svm = LinearSVC(random_state=17)
svm_params = {'C': np.linspace(1e-2, 1, 30)}

svm_grid_searcher = GridSearchCV(svm, svm_params, n_jobs=-1, cv=skf, return_train_score=True)
svm_grid_searcher.fit(X_train_cv, y_train_ss)

CPU times: user 493 ms, sys: 65.1 ms, total: 558 ms
Wall time: 9.21 s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=17, shuffle=True),
             error_score='raise-deprecating',
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=17, tol=0.0001, verbose=0),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([0.01...
       0.18068966, 0.21482759, 0.24896552, 0.28310345, 0.31724138,
       0.35137931, 0.38551724, 0.41965517, 0.4537931 , 0.48793103,
       0.52206897, 0.5562069 , 0.59034483, 0.62448276, 0.65862069,
       0.69275862, 0.72689655, 0.76103448, 0.79517241, 0.82931034,
       0.86344828, 0.89758621, 0.93172414, 0.96586207, 1.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, ver

In [329]:
i = np.argmax(svm_grid_searcher.cv_results_['mean_test_score'])
accu_train = svm_grid_searcher.cv_results_['mean_test_score'][i]
print('best score is {:.5f} correspond to C={} with index {}'.format(accu_train, svm_grid_searcher.cv_results_['param_C'][i], i))

best score is 0.96388 correspond to C=0.14655172413793105 with index 4


Ещё хуже, попробуем на полной выборке логистическую регрессию

In [334]:
X_train_cv, X_test_cv = prepare_data_ext(train_df.iloc[:, :-1], test_df.iloc[:, :-1])
train_share = int(.7 * len(train_df))
X_train, y_train = X_train_cv.tocsr()[:train_share, :], train_df.target[:train_share]
X_valid, y_valid  = X_train_cv.tocsr()[train_share:, :], train_df.target[train_share:]

In [335]:
logit = LogisticRegression(random_state=17, C=7.6316, n_jobs=-1)
logit.fit(X_train, y_train)
logit_valid_pred_proba = logit.predict_proba(X_valid)
roc_auc = roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])
print('ROC AUC for CountVectorizer + 2 feature with LogisticRegression is {:.5f}'.format(roc_auc))

ROC AUC for CountVectorizer + 2 feature with LogisticRegression is 0.96854


Да, действительно всё только хуже

## Критерии оценки работы (только для Peer Review в специализации):
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`). Это только в случае, если вы проходите специализацию.
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.